# 크롤링 연습 : 웹페이지 게시글 제목 크롤링
- 98개의 게시물 제목 크롤링해서 데이터프레임으로 만들기
- 게시물 제목 예 : 서울시 차 없는 거리 효과 분석과 우선 입지 선정 [2022 빅데이터캠퍼스 공모전 - 우수상]
   - 제목 + [연도 공모전명 수상내용] 으로 구성됨

In [46]:
# 테스트
from bs4 import BeautifulSoup
import requests
from lxml import html
import csv
import pandas as pd

url = "https://bigdata.seoul.go.kr/noti/selectPageListTabNoti.do?r_id=P260&bbs_seq=&ac_type=A1&sch_type=&sch_text=&currentPage=1"

response = requests.get(url)
print(response.status_code)
tree = html.fromstring(response.text)

post_titles = tree.xpath('//*[@id="aform"]/div[1]/div/table/tbody/tr[1]/td[2]/a/@title')
print(post_titles)

200
['지하철 9호선 신설역 수요인원 예측모델 개발 및 정책 제안 [2022 빅데이터캠퍼스 공모전 - 서울창조경제혁신센터장상 수상]']


In [47]:
import requests
from lxml import html
import pandas as pd

# 빈 데이터프레임 생성(제목 저장)
df = pd.DataFrame(columns = ['Title'])

for page_number in range(1, 11):
    url = f"https://bigdata.seoul.go.kr/noti/selectPageListTabNoti.do?r_id=P260&bbs_seq=&ac_type=A1&sch_type=&sch_text=&currentPage={page_number}"
    response = requests.get(url)
    
    tree = html.fromstring(response.text)

    # XPath를 사용하여 모든 게시물 제목 선택
    page_post_titles = tree.xpath('//*[@id="aform"]/div[1]/div/table/tbody/tr/td[2]/a/@title')

    # df 에 저장
    df = pd.concat([df, pd.DataFrame({'Title':page_post_titles})], ignore_index = True)

df

,Title
0,지하철 9호선 신설역 수요인원 예측모델 개발 및 정책 제안 [2022 빅데이터캠퍼스...
1,도시데이터 센서 환경정보 기반의 금천구 폭염 취약지역 탐색 [2022 빅데이터캠퍼스...
2,탄소중립 선도를 위한 서울시 종합에너지스테이션 우선 입지 선정 [2022 빅데이터캠...
3,서울시 차 없는 거리 효과 분석과 우선 입지 선정 [2022 빅데이터캠퍼스 공모전 ...
4,환경형평성을 고려한 서울시 내 그린 인프라 조성 [2022 빅데이터캠퍼스 공모전 -...
...,...
93,지하철 역 추가 설치 제안 - 소득수준의 관점에서 [2017 빅캠 상시공모전]
94,서울시 자전거 이동경로 데이터 분석 [2017 빅캠 상시공모전]
95,빅데이터 분석을 통한 서울시 숙박업소 실태와 관광활성화 방안 [2017 빅캠 상시공...
96,서울시 미세먼지 데이터를 활용한 분석 기획:유소년 보행 환경 개선에 대한 시나리오 ...


In [48]:
df.shape

(98, 1)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Title   98 non-null     object
dtypes: object(1)
memory usage: 912.0+ bytes


In [65]:
df['Main_Title'] = df['Title'].str.split('[').str[0].str.strip()
#  '[' 다음에 연속된 숫자를 추출
df['Year'] = df['Title'].str.extract(r'\[([\d]+)')

# '-'와 ']' 사이의 모든 문자열을 추출
df['Award'] = df['Title'].str.extract(r' - ([^\]]+)')

In [66]:
df.head()

,Title,Main_Title,Year,Award
0,지하철 9호선 신설역 수요인원 예측모델 개발 및 정책 제안 [2022 빅데이터캠퍼스...,지하철 9호선 신설역 수요인원 예측모델 개발 및 정책 제안,2022,서울창조경제혁신센터장상 수상
1,도시데이터 센서 환경정보 기반의 금천구 폭염 취약지역 탐색 [2022 빅데이터캠퍼스...,도시데이터 센서 환경정보 기반의 금천구 폭염 취약지역 탐색,2022,최우수상
2,탄소중립 선도를 위한 서울시 종합에너지스테이션 우선 입지 선정 [2022 빅데이터캠...,탄소중립 선도를 위한 서울시 종합에너지스테이션 우선 입지 선정,2022,최우수상
3,서울시 차 없는 거리 효과 분석과 우선 입지 선정 [2022 빅데이터캠퍼스 공모전 ...,서울시 차 없는 거리 효과 분석과 우선 입지 선정,2022,우수상
4,환경형평성을 고려한 서울시 내 그린 인프라 조성 [2022 빅데이터캠퍼스 공모전 -...,환경형평성을 고려한 서울시 내 그린 인프라 조성,2022,우수상


In [67]:
df.drop('Title', axis = 1, inplace = True)

In [68]:
df

,Main_Title,Year,Award
0,지하철 9호선 신설역 수요인원 예측모델 개발 및 정책 제안,2022,서울창조경제혁신센터장상 수상
1,도시데이터 센서 환경정보 기반의 금천구 폭염 취약지역 탐색,2022,최우수상
2,탄소중립 선도를 위한 서울시 종합에너지스테이션 우선 입지 선정,2022,최우수상
3,서울시 차 없는 거리 효과 분석과 우선 입지 선정,2022,우수상
4,환경형평성을 고려한 서울시 내 그린 인프라 조성,2022,우수상
...,...,...,...
93,지하철 역 추가 설치 제안 - 소득수준의 관점에서,2017,소득수준의 관점에서 [2017 빅캠 상시공모전
94,서울시 자전거 이동경로 데이터 분석,2017,NaN
95,빅데이터 분석을 통한 서울시 숙박업소 실태와 관광활성화 방안,2017,NaN
96,서울시 미세먼지 데이터를 활용한 분석 기획:유소년 보행 환경 개선에 대한 시나리오,2017,NaN


In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98 entries, 0 to 97
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Main_Title  98 non-null     object
 1   Year        98 non-null     object
 2   Award       65 non-null     object
dtypes: object(3)
memory usage: 2.4+ KB


In [80]:
df.sort_values(by = ['Award'])

,Main_Title,Year,Award
17,서울시 수소차 충전소 우선 입지 선정,2021,CJ올리브네트웍스상
19,서울특별시 소규모 도심형 물류센터 입지분석,2021,대상
21,서울시 초등돌봄교실 공급 확대 우선 지역 선정,2020,서울창조경제혁신센터장상
18,장애인 포용성 지수 개발을 통한 서울시 내 살기 좋은 지원 주택 추천,2021,서울창조경제혁신센터장상
0,지하철 9호선 신설역 수요인원 예측모델 개발 및 정책 제안,2022,서울창조경제혁신센터장상 수상
...,...,...,...
92,서울시 심야버스 이용 행태 분석,2017,NaN
94,서울시 자전거 이동경로 데이터 분석,2017,NaN
95,빅데이터 분석을 통한 서울시 숙박업소 실태와 관광활성화 방안,2017,NaN
96,서울시 미세먼지 데이터를 활용한 분석 기획:유소년 보행 환경 개선에 대한 시나리오,2017,NaN


In [83]:
df.to_csv('BigdataContest.csv', index = False, encoding = 'utf-8')